In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
!ls -r ../input/

grasp-and-lift-eeg-detection  eegprediction


In [4]:
train_df  = pd.read_csv("../input/eegprediction/train_predicted.csv")

In [5]:
test_df = pd.read_csv("../input/eegprediction/test_predicted.csv")

In [6]:
train_df[:10]

,end,path,series,start,subject,target_frame_end,target_frame_start,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
0,107925,output/features/train/window_750_stride_75/1/1...,1,107175,1,108675,107925,-4.333508,-8.702975,-8.607170,-7.154396,-8.061901,-8.217526
1,41100,output/features/train/window_750_stride_75/1/1...,1,40350,1,41850,41100,-5.718880,-7.249228,-8.459538,-8.829724,1.832593,0.907764
2,43425,output/features/train/window_750_stride_75/1/1...,1,42675,1,44175,43425,-7.539823,-6.328394,-4.137625,-0.804625,-10.299252,-8.672769
3,32475,output/features/train/window_750_stride_75/1/1...,1,31725,1,33225,32475,-3.037788,0.031303,-0.542431,-2.543895,-4.954531,-4.151911
4,71025,output/features/train/window_750_stride_75/1/1...,1,70275,1,71775,71025,-3.069319,-7.836869,-8.112086,-7.207323,-7.000421,-7.407041
5,69525,output/features/train/window_750_stride_75/1/1...,1,68775,1,70275,69525,-3.097936,-9.703127,-9.911842,-8.372626,-8.361329,-9.133985
6,68250,output/features/train/window_750_stride_75/1/1...,1,67500,1,69000,68250,-4.506536,0.883574,1.032072,-0.270910,-6.508389,-5.185030
7,36375,output/features/train/window_750_stride_75/1/1...,1,35625,1,37125,36375,-6.110576,1.007050,1.655226,0.807872,-8.873638,-6.981388
8,28950,output/features/train/window_750_stride_75/1/1...,1,28200,1,29700,28950,-2.054430,-6.761213,-6.958857,-6.044637,-6.355172,-6.782236
9,98325,output/features/train/window_750_stride_75/1/1...,1,97575,1,99075,98325,-4.949322,-8.478207,-9.694247,-9.448436,0.948595,-1.036801


## apply sigmoid

In [7]:
from pathlib import Path

In [8]:
import torch

In [9]:
for col in train_df.columns[-6:]:
    train_df[col] = torch.sigmoid(torch.Tensor(train_df[col].values)).numpy()

In [10]:
for col in test_df.columns[-6:]:
    test_df[col] = torch.sigmoid(torch.Tensor(test_df[col].values)).numpy()

In [11]:
train_df[:10]

,end,path,series,start,subject,target_frame_end,target_frame_start,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
0,107925,output/features/train/window_750_stride_75/1/1...,1,107175,1,108675,107925,0.012951,0.000166,0.000183,0.000781,0.000315,0.000270
1,41100,output/features/train/window_750_stride_75/1/1...,1,40350,1,41850,41100,0.003273,0.000710,0.000212,0.000146,0.862070,0.712542
2,43425,output/features/train/window_750_stride_75/1/1...,1,42675,1,44175,43425,0.000531,0.001782,0.015710,0.309037,0.000034,0.000171
3,32475,output/features/train/window_750_stride_75/1/1...,1,31725,1,33225,32475,0.045748,0.507825,0.367622,0.072838,0.007002,0.015491
4,71025,output/features/train/window_750_stride_75/1/1...,1,70275,1,71775,71025,0.044391,0.000395,0.000300,0.000741,0.000911,0.000607
5,69525,output/features/train/window_750_stride_75/1/1...,1,68775,1,70275,69525,0.043192,0.000061,0.000050,0.000231,0.000234,0.000108
6,68250,output/features/train/window_750_stride_75/1/1...,1,67500,1,69000,68250,0.010916,0.707562,0.737317,0.432684,0.001489,0.005569
7,36375,output/features/train/window_750_stride_75/1/1...,1,35625,1,37125,36375,0.002214,0.732442,0.839596,0.691656,0.000140,0.000928
8,28950,output/features/train/window_750_stride_75/1/1...,1,28200,1,29700,28950,0.113605,0.001156,0.000949,0.002365,0.001735,0.001132
9,98325,output/features/train/window_750_stride_75/1/1...,1,97575,1,99075,98325,0.007038,0.000208,0.000062,0.000079,0.720832,0.261768


In [12]:
train_df.to_csv("train_predicted_sigmoid.csv", index=None, encoding="utf-8")

In [13]:
test_df[:10]

,end,path,series,start,subject,target_frame_end,target_frame_start,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
0,28350,output/features/test/window_750_stride_75/1/9/...,9,27600,1,29100,28350,0.006017,1.052854e-02,3.287561e-02,1.619377e-01,7.485446e-04,1.947649e-03
1,31275,output/features/test/window_750_stride_75/1/9/...,9,30525,1,32025,31275,0.060642,4.864316e-04,3.627918e-04,9.363033e-04,2.275195e-03,1.962346e-03
2,9825,output/features/test/window_750_stride_75/1/9/...,9,9075,1,10575,9825,0.006214,5.395806e-01,3.209297e-01,1.873735e-02,3.153130e-04,1.285488e-03
3,23475,output/features/test/window_750_stride_75/1/9/...,9,22725,1,24225,23475,0.001953,3.567692e-08,3.650340e-08,9.014056e-07,1.841217e-07,7.902177e-08
4,10125,output/features/test/window_750_stride_75/1/9/...,9,9375,1,10875,10125,0.000791,2.408809e-02,1.335545e-01,5.775644e-01,4.620197e-05,1.917336e-04
5,5925,output/features/test/window_750_stride_75/1/9/...,9,5175,1,6675,5925,0.014761,5.706047e-04,4.055520e-04,8.349184e-04,9.333073e-03,7.132459e-03
6,28800,output/features/test/window_750_stride_75/1/9/...,9,28050,1,29550,28800,0.002588,1.055784e-04,1.987618e-04,1.606193e-03,1.524443e-04,2.407080e-04
7,37575,output/features/test/window_750_stride_75/1/9/...,9,36825,1,38325,37575,0.002489,8.514872e-05,3.132899e-05,4.796389e-05,3.492737e-01,2.240100e-01
8,101550,output/features/test/window_750_stride_75/1/9/...,9,100800,1,102300,101550,0.000665,1.545213e-05,7.137701e-05,2.590819e-03,4.307316e-06,8.750584e-06
9,77775,output/features/test/window_750_stride_75/1/9/...,9,77025,1,78525,77775,0.004277,1.249607e-07,1.065433e-07,1.990287e-06,4.020613e-06,2.029391e-06


In [14]:
test_df.to_csv("test_predicted_sigmoid.csv", index=None, encoding="utf-8")

In [15]:
train_raw_data_dir = Path("../input/grasp-and-lift-eeg-detection/train/")

##  convert window prediction to frame prediction

In [16]:
train_df.sort_values(by=["subject", "series", "target_frame_start"], axis=0, inplace=True)

In [17]:
train_df = train_df.reset_index()

In [18]:
train_df[:10]

,index,end,path,series,start,subject,target_frame_end,target_frame_start,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
0,156,750,output/features/train/window_750_stride_75/1/1...,1,0,1,1500,750,0.016679,0.008458,0.006750,0.008186,0.049698,0.051856
1,1132,825,output/features/train/window_750_stride_75/1/1...,1,75,1,1575,825,0.007936,0.003499,0.002614,0.003569,0.078341,0.100869
2,1038,900,output/features/train/window_750_stride_75/1/1...,1,150,1,1650,900,0.002049,0.002350,0.005426,0.028799,0.002393,0.006800
3,618,975,output/features/train/window_750_stride_75/1/1...,1,225,1,1725,975,0.000590,0.000323,0.001215,0.017537,0.000339,0.001379
4,207,1050,output/features/train/window_750_stride_75/1/1...,1,300,1,1800,1050,0.000158,0.000151,0.001520,0.069867,0.000005,0.000022
5,511,1125,output/features/train/window_750_stride_75/1/1...,1,375,1,1875,1125,0.000231,0.000013,0.000061,0.002223,0.000017,0.000067
6,1392,1200,output/features/train/window_750_stride_75/1/1...,1,450,1,1950,1200,0.000987,0.000002,0.000003,0.000049,0.000059,0.000104
7,1176,1275,output/features/train/window_750_stride_75/1/1...,1,525,1,2025,1275,0.000659,0.000001,0.000003,0.000056,0.000019,0.000027
8,449,1350,output/features/train/window_750_stride_75/1/1...,1,600,1,2100,1350,0.000817,0.000001,0.000001,0.000017,0.000024,0.000024
9,470,1425,output/features/train/window_750_stride_75/1/1...,1,675,1,2175,1425,0.002578,0.000005,0.000003,0.000016,0.000086,0.000072


In [19]:
test_df.sort_values(by=["subject", "series", "target_frame_start"], axis=0, inplace=True)

In [20]:
test_df = test_df.reset_index()

In [21]:
train_events = list(train_raw_data_dir.glob("*events.csv"))

In [22]:
event_df_sample = pd.read_csv(train_events[0], encoding="utf-8")

In [23]:
STRIDE_SIZE = 75

In [24]:
OFFSET = 750

In [25]:
event_df_sample[:10]

,id,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
0,subj12_series7_0,0,0,0,0,0,0
1,subj12_series7_1,0,0,0,0,0,0
2,subj12_series7_2,0,0,0,0,0,0
3,subj12_series7_3,0,0,0,0,0,0
4,subj12_series7_4,0,0,0,0,0,0
5,subj12_series7_5,0,0,0,0,0,0
6,subj12_series7_6,0,0,0,0,0,0
7,subj12_series7_7,0,0,0,0,0,0
8,subj12_series7_8,0,0,0,0,0,0
9,subj12_series7_9,0,0,0,0,0,0


In [26]:
train_frame_df = pd.DataFrame(columns=event_df_sample.columns)

In [27]:
train_df.shape

(239763, 14)

In [28]:
def from_window_to_frame(row):
    start_frame = row[1]["target_frame_start"] - OFFSET
    frame_dicts = []
    for i in range(STRIDE_SIZE):
        frame = start_frame + i
        id_ = "subj{}_series{}_{}".format(row[1]["subject"], row[1]["series"], frame)
        frame_dict = {col: row[1][col] for col in event_df_sample.columns[-6:]}
        frame_dict["id"] = id_
        frame_dicts.append(frame_dict)
    return frame_dicts

In [29]:
from tqdm import tqdm_notebook

In [30]:
from multiprocessing.pool import Pool

In [31]:
from sklearn.externals.joblib import Parallel, delayed

In [32]:
with Pool() as pool:
    train_frame_df = list(pool.map(from_window_to_frame, list(train_df.iterrows())))

In [33]:
import itertools

In [34]:
train_frame_df = pd.DataFrame.from_records(
    itertools.chain.from_iterable(train_frame_df))

In [35]:
train_frame_df.to_csv("train_frame_prediction.csv")

In [36]:
with Pool() as pool:
    test_frame_df = list(pool.map(from_window_to_frame, list(test_df.iterrows())))

In [37]:
test_frame_df = pd.DataFrame.from_records(
    itertools.chain.from_iterable(test_frame_df))

In [38]:
test_frame_df.to_csv("test_frame_prediction.csv")

In [39]:
# test_frame_df = Parallel(n_jobs=-1, verbose=1)([delayed(from_window_to_frame)(row) for row 
#                                  in list(test_df.iterrows())[:10]])

In [40]:
# train_frame_df = joblib.Parallel(n_jobs=-1, verbose=1)([joblib.delayed(from_window_to_frame)(roundw) for row 
#                                  in train_df.iterrows()])

In [41]:
train_frame_df[:10]

,BothReleased,BothStartLoadPhase,FirstDigitTouch,HandStart,LiftOff,Replace,id
0,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_0
1,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_1
2,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_2
3,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_3
4,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_4
5,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_5
6,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_6
7,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_7
8,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_8
9,0.051856,0.00675,0.008458,0.016679,0.008186,0.049698,subj1_series1_9
